# Training Embeddings con GNN
Aca crearemos embeddings paa un grafo de Internet, es decir represenaciones de los SA a partir de la topologia y atributos de los SA.

Para esto tomamos 3 enfoques:


*   [Caso 1] Reconstruction Approach autoencoder: A traves de la prediccion de aristas construimos los embeddings de los nodos.
*   [Caso 1.1] Reconstruction Approach autoencoder: en ves de ocupar las bgp routes recolectadas de los ribs ocupamos las relaciones de AS rank y le damso attr a las aristas (es solo para probar que lo estemos ahciendo bien y las cosas tienen sentido)
*   [Caso 2]: Reconstruction Approach attribute Masking:
*   Caso 3: Task Generation Pre-calculated descriptor

Para cada uno crearemos un ejemplo.


In [131]:
%load_ext autoreload
%autoreload 2
# from modules.gnn import GNN
import scipy.sparse as sp
import numpy as np
import os

from modules.graph import Graph, create_files
from modules.gnn import GNN
from modules.gnn_models import GCN, GraphSAGE

import torch
import torch.nn.functional as F

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [132]:
# Definimos las rutas de los archivos
base_path = os.getcwd() + "/data/"
relationships_file = base_path + "CAIDA_AS_Relationships/Serial_1/20220701.as-rel.txt.bz2"
features_file = base_path + "/node_features.csv"
rib_path = base_path + 'sanitized_rib.txt'
dataset_graph_path = base_path + 'dgl_graph/'

## Creacion Grafo 
Creamos un grafo nx y dgl, ademas de los archivos edges.csv y nodes.csv a partir de archivos ribs previamente creados o de archivo CAIDA AS Relationships.

Crear esos archivos una unica vez con create_graph() una ves ya creados los archivos edges.csv y nodes.csv puedo ocupar directamente la funcion 

Se le puede indicar el maximo de bgp paths que se quiere (hehco para cuando se leen ribs no de caida) 

### CASO 1: RIBs
* Creacion de grafo a partir de paths recolectados de las RIBs por BGPStream
* Por ahora le asignamos a todos los nodos embeddings iniciales de de dimension 32 parte con puros 1s todos


In [152]:
# Definimos las listas de features

# features_file = 'node_degrees' #'' # 'node_degrees' # las features que se le agregaran seran 
# features_file = base_path + "/node_features.csv"
features_file = base_path + "/node_features_mio.csv"
type = 'MultiDiGraph'#"DiGraph" # MultiDiGraph
relationships_file
MAX_PATHS = 100000
graph_case1 = create_files(type, 
            dataset_graph_path,
            file = rib_path, 
            features_file = features_file, 
            from_caida=False, 
            remove_degree=3,
            debug=True,
            max_paths = MAX_PATHS)
 
print('[NX Graph]',graph_case1.nx_graph)

[CaRPETA CREADA]:  /home/valentina/Desktop/GIT/TrabajoTesis/data/dgl_graph/
[Creando topologia]
[ARCHIVO EDGES.CSV CREADO]
[NX Graph]:  MultiDiGraph with 36580 nodes and 325860 edges
[Agregando attr]
[NX Graph]:  MultiDiGraph with 36580 nodes and 325860 edges
nodes.csv edges.csv
[FEATURES CREADOS]
[META CREADO]
[NX Graph]:  MultiDiGraph with 26584 nodes and 315864 edges
[NX Graph]:  MultiDiGraph with 26495 nodes and 315775 edges
[NX Graph]:  MultiDiGraph with 26494 nodes and 315774 edges
[NX Graph] MultiDiGraph with 26494 nodes and 315774 edges


### CASO 2: CAIDA Relationships
* Creacion del grafo a partir de CAIDA AS Relationships (AS Rank) 
* Se les da atributos a los edges correspondientes al tipo de relacion que comparten
* Es solo de prueba 

In [51]:
# Definimos las listas de features

features_file =  'data/node_features.csv'  #'node_degrees' #'' # 'node_degrees' # las features que se le agregaran seran 
type = "DiGraph" # MultiDiGraph

max_paths = 100000
graph_case1 = create_files(type, 
             dataset_graph_path,
             relationships_file, 
             features_file, 
             from_caida=True, 
             remove_degree=3,
             debug=True,
max_paths = None)
 
print('[NX Graph]',graph_case1.nx_graph)

[CaRPETA CREADA]:  /home/valentina/Desktop/GIT/TrabajoTesis/data/dgl_graph/
[NX Graph]:  DiGraph with 74140 nodes and 379420 edges
[TOPOLOGIA CREADA]
[todos attr]
[bbbbbbbbbbbbbbbbbbbbbb]
[NX Graph]:  DiGraph with 74140 nodes and 379420 edges
nodes.csv edges.csv
[FEATURES CREADOS]
[META CREADO]
[NX Graph]:  DiGraph with 46714 nodes and 351994 edges
[NX Graph]:  DiGraph with 46237 nodes and 351517 edges
[NX Graph]:  DiGraph with 46229 nodes and 351509 edges
[NX Graph] DiGraph with 46229 nodes and 351509 edges


## Link Prediction: Encode-Decoder 

In [101]:
import dgl
import torch
import torch.nn as nn
import torch.nn.functional as F
from dgl.data import CoraGraphDataset
from dgl.dataloading import negative_sampler
import dgl.function as fn
from sklearn.metrics import roc_auc_score
from dgl.nn import SAGEConv, GraphConv, GATConv, GINConv, GatedGCNConv, GatedGraphConv

import numpy as np
import scipy.sparse as sp
from sklearn.metrics import roc_auc_score



from dgl.nn import SAGEConv
from dgl.nn import SAGEConv, GraphConv, GATConv, GINConv, GatedGCNConv, GatedGraphConv
import torch
import torch.nn as nn
import torch.nn.functional as F
import itertools


In [103]:
def compute_auc(pos_score,neg_score):
    scores = torch.cat([pos_score, neg_score]).numpy()
    labels = torch.cat([torch.ones(pos_score.shape[0]), torch.zeros(neg_score.shape[0])]).numpy()
    return roc_auc_score(labels, scores)


def compute_loss(pos_score, neg_score):
    scores = torch.cat([pos_score, neg_score])
    labels = torch.cat(
        [torch.ones(pos_score.shape[0]), torch.zeros(neg_score.shape[0])]
    )
    return F.binary_cross_entropy_with_logits(scores, labels)


### CASO 1: 
* Encoder : GNN
* Decoder : DotProduct

In [ ]:
gnn = GNN(debug=True)
gnn.load_dataset(dataset_graph_path, force_reload=True)

print('[ATTR SHAPE]: ',gnn.dgl_graph.ndata['feat'].shape)

gnn.split_graph_edges(train_size=0.8)

Done saving data into cached files.
Graph(num_nodes=36580, num_edges=315774,
      ndata_schemes={'feat': Scheme(shape=(69,), dtype=torch.float32)}
      edata_schemes={'Relationship': Scheme(shape=(), dtype=torch.float32)})
[ATTR SHAPE]:  torch.Size([36580, 69])
Generando 315774 aristas negativas...
Aristas negativas generadas: 315773


In [105]:

model = GraphSAGE(gnn.train_g.ndata["feat"].shape[1], 16,16)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

# ----------- 4. training -------------------------------- #

all_logits = []
for e in range(100):
    # forward
    model.train()
    h = model.encode(gnn.train_g, gnn.train_g.ndata["feat"])

    pos_score = model.decodeDotProduct(gnn.train_pos_g, h)
    neg_score = model.decodeDotProduct(gnn.train_neg_g, h)


    loss = compute_loss(pos_score, neg_score)

    # backward
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    if e % 5 == 0:
        print("In epoch {}, loss: {}".format(e, loss))

# ----------- 5. check results ------------------------ #
from sklearn.metrics import roc_auc_score

with torch.no_grad():
    pos_score = model.decodeDotProduct(gnn.test_pos_g, h)
    neg_score = model.decodeDotProduct(gnn.test_neg_g, h)
    print("AUC", compute_auc(pos_score, neg_score))


In epoch 0, loss: 2.3666417598724365
In epoch 5, loss: 0.6438419818878174
In epoch 10, loss: 0.5738990306854248
In epoch 15, loss: 0.5256621241569519
In epoch 20, loss: 0.5031626224517822
In epoch 25, loss: 0.4877111613750458
In epoch 30, loss: 0.47665834426879883
In epoch 35, loss: 0.46760863065719604
In epoch 40, loss: 0.46020612120628357
In epoch 45, loss: 0.45447930693626404
In epoch 50, loss: 0.4497029781341553
In epoch 55, loss: 0.4458879828453064
In epoch 60, loss: 0.4427325427532196
In epoch 65, loss: 0.44007453322410583
In epoch 70, loss: 0.43783459067344666
In epoch 75, loss: 0.43594813346862793
In epoch 80, loss: 0.434280127286911
In epoch 85, loss: 0.4327625334262848
In epoch 90, loss: 0.4313984811306
In epoch 95, loss: 0.4301387667655945
AUC 0.9432483659934531


In [107]:
# Guaradar el modelo
torch.save(model.state_dict(), 'data/model_emb.pth')

# Guardar los embeddings
torch.save(h, "data/embeddings_ribs_DP.pt")


### CASO 2: 
* Encoder : GNN
* Decoder : MLP

In [134]:
gnn = GNN(debug=True)
gnn.load_dataset(dataset_graph_path, force_reload=True)

# FIXME: Cambiar, por mientras se estan agregando feat aleatorias o 1s a los nodos
# gnn.dgl_graph.ndata['feat'] = torch.ones(gnn.dgl_graph.num_nodes(), 128)            # features de tamano 128

print('[ATTR SHAPE]: ',gnn.dgl_graph.ndata['feat'].shape)

gnn.split_graph_edges(train_size=0.8)


Done saving data into cached files.
Graph(num_nodes=36580, num_edges=315774,
      ndata_schemes={'feat': Scheme(shape=(69,), dtype=torch.float32)}
      edata_schemes={'Relationship': Scheme(shape=(), dtype=torch.float32)})
[ATTR SHAPE]:  torch.Size([36580, 69])
Generando 315774 aristas negativas...
Aristas negativas generadas: 315773


In [135]:
model = GraphSAGE(gnn.train_g.ndata["feat"].shape[1], 16,16)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

# ----------- 4. training -------------------------------- #

all_logits = []
for e in range(50):
    # forward
    model.train()
    h = model.encode(gnn.train_g, gnn.train_g.ndata["feat"])

    pos_score = model.decodeMLP(gnn.train_pos_g, h)
    neg_score = model.decodeMLP(gnn.train_neg_g, h)


    loss = compute_loss(pos_score, neg_score)

    # backward
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    if e % 5 == 0:
        print("In epoch {}, loss: {}".format(e, loss))

# ----------- 5. check results ------------------------ #
from sklearn.metrics import roc_auc_score

with torch.no_grad():
    pos_score = model.decodeMLP(gnn.test_pos_g, h)
    neg_score = model.decodeMLP(gnn.test_neg_g, h)
    print("AUC", compute_auc(pos_score, neg_score))

In epoch 0, loss: 0.6792717576026917
In epoch 5, loss: 0.42843642830848694
In epoch 10, loss: 0.2641475796699524
In epoch 15, loss: 0.20924416184425354
In epoch 20, loss: 0.17611178755760193
In epoch 25, loss: 0.15358121693134308
In epoch 30, loss: 0.1409989595413208
In epoch 35, loss: 0.1279410719871521
In epoch 40, loss: 0.1201382726430893
In epoch 45, loss: 0.1134682297706604
AUC 0.9879080912494592


In [137]:
# Se incluyen los parametros tanto de MLP como de la GNN

for name, param in model.named_parameters():
    print(name, param.shape)


conv1.fc_neigh.weight torch.Size([16, 69])
conv1.fc_self.weight torch.Size([16, 69])
conv1.fc_self.bias torch.Size([16])
conv2.fc_neigh.weight torch.Size([16, 16])
conv2.fc_self.weight torch.Size([16, 16])
conv2.fc_self.bias torch.Size([16])
decoder.W1.weight torch.Size([16, 32])
decoder.W1.bias torch.Size([16])
decoder.W2.weight torch.Size([1, 16])
decoder.W2.bias torch.Size([1])


#### Guardar Modelo y Embeddings

In [138]:
# Guaradar el modelo
torch.save(model.state_dict(), 'data/model_emb.pth')

# Guardar los embeddings
torch.save(h, "data/embeddings_ribs_MLP.pt")


Cosas a tener consideraciom:
- GNN al crear un grafo, crea tdoos los nodos que se encuentran en el rango entregado. De esta forma pueden quedar nodos aislados, sin embargo ento causa que no se puedan ir 'actualizando' pero al agregar self_loop se actualizan con si mismos y supondremos que con ello algun patron para este tipo de nodos.
- La entrega de embeddings finales esta en orden el ASN de valor inferios hasta el ASN de valor maximo.
- Lo guardamso en un .csv dondde se asocia 'node_id' con su embeddings y donde node_id es el ASN del Sistema Autonomo. 

## Inferencia valor PAgeRank


In [153]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import dgl

class GNNPredictLastFeat(nn.Module):
    def __init__(self, in_feats, hidden_feats):
        super().__init__()
        self.conv1 = dgl.nn.GraphConv(in_feats, hidden_feats)
        self.conv2 = dgl.nn.GraphConv(hidden_feats, hidden_feats)
        self.regressor = nn.Linear(hidden_feats, 1)  # solo predice 1 número por nodo

    def forward(self, g, features):
        h = F.relu(self.conv1(g, features))
        h = F.relu(self.conv2(g, h))
        out = self.regressor(h)  # predicción de un valor
        return out.squeeze(-1)   # tamaño [n_nodos]


In [160]:
gnn = GNN(debug=True)
gnn.load_dataset(dataset_graph_path, force_reload=True)


Done saving data into cached files.
Graph(num_nodes=36580, num_edges=315774,
      ndata_schemes={'feat': Scheme(shape=(69,), dtype=torch.float32)}
      edata_schemes={'Relationship': Scheme(shape=(), dtype=torch.float32)})


In [161]:
# Quitar valor a inferir
feats = gnn.dgl_graph.ndata['feat']
targets = feats[:, -1]
new_feats = feats[:, :-1]

gnn.dgl_graph.ndata['feat'] = new_feats
gnn.dgl_graph = dgl.add_self_loop(gnn.dgl_graph) # FIXME:

gnn.split_graph_nodes(train_size=0.8)

                                

In [167]:

train_mask = gnn.dgl_graph.train_mask
test_mask = gnn.dgl_graph.test_mask

print(train_mask)
print(test_mask)

tensor([False,  True,  True,  ...,  True, False, False])
tensor([ True, False, False,  ..., False,  True,  True])


In [183]:
# -------------- Entrenamiento ------------------ #
import torch.optim as optim

# Creamos el modelo
model = GNNPredictLastFeat(in_feats=gnn.dgl_graph.ndata['feat'].shape[1], hidden_feats=64)

# Forward pass


optimizer = optim.Adam(model.parameters(), lr=0.01)
loss_fn = nn.MSELoss()  # Porque es regresión

for epoch in range(40):
    model.train()
    logits = model(gnn.dgl_graph, gnn.dgl_graph.ndata['feat'])  # predictions.shape = [n_nodes]
    loss = F.mse_loss(logits[train_mask], targets[train_mask])    
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    print(f"Epoch {epoch}: Loss = {loss.item():.4f}")

# model.eval()
# with torch.no_grad():
#     pred = model(gnn.dgl_graph, gnn.dgl_graph.ndata['feat'])
    
#     pred = (pred[test_mask] > 0.5).float()  # Convertir a 0 o 1
#     acc = (pred == targets[test_mask]).float().mean()
#     print(f"Test Accuracy: {acc.item():.4f}")


Epoch 0: Loss = 0.0177
Epoch 1: Loss = 0.0045
Epoch 2: Loss = 0.0078
Epoch 3: Loss = 0.0035
Epoch 4: Loss = 0.0010
Epoch 5: Loss = 0.0005
Epoch 6: Loss = 0.0004
Epoch 7: Loss = 0.0004
Epoch 8: Loss = 0.0003
Epoch 9: Loss = 0.0004
Epoch 10: Loss = 0.0004
Epoch 11: Loss = 0.0003
Epoch 12: Loss = 0.0001
Epoch 13: Loss = 0.0001
Epoch 14: Loss = 0.0001
Epoch 15: Loss = 0.0001
Epoch 16: Loss = 0.0001
Epoch 17: Loss = 0.0001
Epoch 18: Loss = 0.0001
Epoch 19: Loss = 0.0001
Epoch 20: Loss = 0.0001
Epoch 21: Loss = 0.0001
Epoch 22: Loss = 0.0000
Epoch 23: Loss = 0.0000
Epoch 24: Loss = 0.0000
Epoch 25: Loss = 0.0000
Epoch 26: Loss = 0.0000
Epoch 27: Loss = 0.0000
Epoch 28: Loss = 0.0000
Epoch 29: Loss = 0.0000
Epoch 30: Loss = 0.0000
Epoch 31: Loss = 0.0000
Epoch 32: Loss = 0.0000
Epoch 33: Loss = 0.0000
Epoch 34: Loss = 0.0000
Epoch 35: Loss = 0.0000
Epoch 36: Loss = 0.0000
Epoch 37: Loss = 0.0000
Epoch 38: Loss = 0.0000
Epoch 39: Loss = 0.0000


In [184]:
model.eval()
with torch.no_grad():
    preds = model(gnn.dgl_graph, gnn.dgl_graph.ndata['feat'])

mse = F.mse_loss(preds[test_mask], targets[test_mask]).item()
print(f"Test MSE: {mse:.4f}")


Test MSE: 0.0000


In [185]:
gnn.dgl_graph.ndata['feat'].shape

torch.Size([36580, 68])

In [186]:
# Guaradar el modelo
torch.save(model.state_dict(), 'data/model_emb.pth')

# Guardar los embeddings
torch.save(h, "data/embeddings_ribs_PageRank.pt")

# Predict otro



# Crear Embeddings otros

## DeepWalk

In [44]:
from dgl.nn import DeepWalk
from torch.utils.data import DataLoader
from torch.optim import SparseAdam

In [ ]:
gnn = GNN(debug=True)
gnn.load_dataset(dataset_graph_path, force_reload=True)

Done saving data into cached files.
Graph(num_nodes=36580, num_edges=315774,
      ndata_schemes={'feat': Scheme(shape=(74,), dtype=torch.float32)}
      edata_schemes={'Relationship': Scheme(shape=(), dtype=torch.float32)})
[ATTR SHAPE]:  torch.Size([36580, 74])
Generando 315774 aristas negativas...
Aristas negativas generadas: 315773


In [64]:
emb = DeepWalk(gnn.dgl_graph, 
               emb_dim=32,        # tamaño del embedding
               walk_length=6,     # número de saltos por walk
            #    context_size=5,     # tamaño de la ventana de contexto
            #    walks_per_node=10,  # número de walks por nodo
            #    num_negative=5
               )     # número de negativos por positivo

emb = DeepWalk(gnn.dgl_graph)

In [ ]:

dataloader = DataLoader(torch.arange(gnn.dgl_graph.num_nodes()),
                        batch_size=500,
                        shuffle=True, 
                        collate_fn=emb.sample,)

optimizer = SparseAdam(emb.parameters(), lr=0.01)

num_epochs = 1

for epoch in range(num_epochs):
    for batch_walk in dataloader:
        # Reemplazar -1 por un nodo válido (ej: nodo 0)
        batch_walk[batch_walk == -1] = 0

        loss = emb(batch_walk)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        

In [67]:
h = emb.node_embed.weight.detach()
print("SHAPE",h.shape)
print("EMBEDDINGS",h)

SHAPE torch.Size([36580, 128])
EMBEDDINGS tensor([[-0.1586, -0.1637,  0.1585,  ...,  0.1625, -0.1555,  0.1694],
        [-0.2038, -0.1939,  0.2022,  ...,  0.1929, -0.1953,  0.1814],
        [-0.1803, -0.1698,  0.1698,  ...,  0.1676, -0.1702,  0.1691],
        ...,
        [-0.0862, -0.0907,  0.0921,  ...,  0.0857, -0.0850,  0.0945],
        [-0.1030, -0.1055,  0.1089,  ...,  0.0963, -0.1074,  0.0997],
        [-0.0967, -0.0872,  0.0959,  ...,  0.0849, -0.0953,  0.0819]])


In [68]:
# Guardar los embeddings
torch.save(h, "data/embeddings_deepWalk.pt")

## PageRank

In [ ]:
gnn = GNN(debug=True)
gnn.load_dataset(dataset_graph_path, force_reload=True)

Done saving data into cached files.
Graph(num_nodes=36580, num_edges=315774,
      ndata_schemes={'feat': Scheme(shape=(74,), dtype=torch.float32)}
      edata_schemes={'Relationship': Scheme(shape=(), dtype=torch.float32)})
[ATTR SHAPE]:  torch.Size([36580, 74])
Generando 315774 aristas negativas...
Aristas negativas generadas: 315773


In [74]:
# ----------- 2. Run PageRank for Graph -------------- #
N = gnn.dgl_graph.number_of_nodes()
DAMP = 0.85
K = 10


def compute_pagerank(g):
    g.ndata["pv"] = torch.ones(N) / N
    degrees = g.out_degrees(g.nodes()).type(torch.float32)
    for k in range(K):
        g.ndata["pv"] = g.ndata["pv"] / degrees
        g.update_all(
            message_func=fn.copy_u(u="pv", out="m"),
            reduce_func=fn.sum(msg="m", out="pv"),
        )
        g.ndata["pv"]  = (1 - DAMP) / N + DAMP * g.ndata["pv"]
    g.ndata["pv"]  = g.ndata["pv"].unsqueeze(1)
    return g.ndata["pv"]


pv = compute_pagerank(gnn.dgl_graph)
gnn.dgl_graph.ndata["h"] = pv  # Inicializa características del nodo

print(gnn.dgl_graph.ndata["h"])

tensor([[1.5807e-05],
        [1.7451e-05],
        [2.1136e-05],
        ...,
        [4.1371e-06],
        [4.3104e-06],
        [4.2975e-06]])


In [77]:
h = gnn.dgl_graph.ndata["h"]
torch.save(h, "data/embeddings_pageRank.pt")